In [1]:
from src.model import CausalDiagram as CD
from src.msbd import MultiOutcomeSequentialBackdoorCriterion as mSBD
from src.sac import SequentialAdjustmentCriterion as SAC


In [2]:
# Fig 1 : Graph exemplifying mSBD. 

G_1 = CD(
    vs = {'X1','X2'} | {'Y1','Y2'} | {'Za','Zb','Zc','Zd'},
    directed_edges =(('Zb', 'Y1'),
    ('Zb', 'X2'),
    ('Zb', 'X1'),
    ('Zd', 'Y2'),
    ('Za', 'Zb'),
    ('Za', 'X1'),
    ('Zc', 'Zd'),
    ('Zc', 'X2'),
    ('X2', 'Y2'),
    ('X1', 'Y1'),
    ('Y1', 'X2')),
    bidirected_edges = [
    ('Zb', 'Zc', 'U0')
    ])

xs,ys,zs = {'X1','X2'},{'Y1','Y2'},{'Za','Zb','Zc','Zd'}

msbd = mSBD(G_1,xs,ys,zs)

# we define Z_1 = {Z_a, Z_b} and Z_2 = {Z_c, Z_d}
msbd.set_covariate(1,{'Za','Zb'})
msbd.set_covariate(2,{'Zc','Zd'})

print('mSBD criterion : ',msbd.criteria())

for e in msbd.G.edges:
    if e in  msbd.localGraph(1).edges:
        pass
    else :
        print(e, 'is removed')

print('-------------------')

for e in msbd.G.edges:
    if e in  msbd.localGraph(2).edges:
        pass
    else :
        print(e, 'is removed')

print('-------------------')

# (mSBD -> SAC)
sac = SAC(G_1,xs,ys,zs)

sac.set_covariate(1,{'Za','Zb'})
sac.set_covariate(2,{'Zc','Zd'})

for e in sac.G.edges:
    if e in  sac.psbd(1).edges:
        pass
    else :
        print(e, 'is removed')

print('-------------------')


for e in sac.G.edges:
    if e in  sac.psbd(2).edges:
        pass
    else :
        print(e, 'is removed')

print('-------------------')


mSBD criterion :  True
('Zb', 'X2') is removed
('Zc', 'X2') is removed
('X1', 'Y1') is removed
('Y1', 'X2') is removed
-------------------
('X2', 'Y2') is removed
-------------------
('Zb', 'X2') is removed
('Zc', 'X2') is removed
('X1', 'Y1') is removed
('Y1', 'X2') is removed
-------------------
('X2', 'Y2') is removed
-------------------


In [3]:
# Fig 2 - SAC example

G_2 = CD(
    vs = {'X1', 'X2', 'Y1', 'Y2', 'Za', 'Zb', 'Zc', 'Zd'},
    directed_edges=(('X2', 'Y2'),
    ('X2', 'Zc'),
    ('Zb', 'X1'),
    ('Zb', 'X2'),
    ('Zb', 'Y1'),
    ('Za', 'Zb'),
    ('Za', 'X1'),
    ('Y1', 'X2'),
    ('X1', 'Y1'),
    ('Zc', 'Zd')),
    bidirected_edges=
        ({('X1', 'X2', 'U2'),
        ('X1', 'Za', 'U1'),
        ('X2', 'Y1', 'U3'),
        ('X2', 'Zb', 'U4'),
        ('X2', 'Zc', 'U5'),
        ('Y2', 'Zd', 'U6'),
        ('Zb', 'Zc', 'U0')}))

xs,ys,zs = {'X1','X2'}, {'Y1','Y2'}, {'Za','Zb','Zc','Zd'}

sac = SAC(G_2,xs,ys,zs)
msbd = mSBD(G_2,xs,ys,zs)

# set Z:=(Z_1,Z_2) where Z_1 = {Z_a, Z_b} and Z_2 = {Z_c, Z_d}
msbd.set_covariate(1,{'Za','Zb'})
msbd.set_covariate(2,{'Zc','Zd'})

# Z_1 satisfies the mSBD criterion
print(msbd.condition1(1),msbd.condition2(1))
# However, Z_2 does not satisfy the mSBD criterion (c1)
print(msbd.condition1(2),msbd.condition2(2))

# Nevertheless, the causal effect is still expressible trough Sequential covariate adjustment

sac.set_covariate(1,{'Za','Zb'})
sac.set_covariate(2,{'Zc','Zd'})

print('SAC criterion : ',sac.criteria())

sac.construct()
print('consturcted Z : ',sac.Z)

True True
False True
SAC criterion :  True
consturcted Z :  [frozenset({'Za', 'Zb'}), frozenset()]


In [4]:
# Fig 3 - Van der Zander 2014 

G_3 = CD(
    vs = {'X1', 'X2', 'Y1', 'Y2', 'Za', 'Zb'},
    directed_edges = (('Zb', 'X2'), ('Za', 'Zb'), ('X1', 'Za'), ('X1', 'Y1'), ('Y2', 'Zb'))
)

xs,ys,zs = {'X1','X2'},{'Y1','Y2'},{'Za','Zb'}

sac = SAC(G_3,xs,ys,zs)
sac.set_covariate(1,{'Za','Zb'})
sac.set_covariate(2,{})

ac = sac.to_ac()

print('AC criterion :',ac.criteria())
print('SAC criterion :',sac.criteria())
print('Z = ',ac.zs)


AC criterion : True
SAC criterion : True
Z =  frozenset({'Za', 'Zb'})


In [6]:
# Fig4

G_4 = CD(
    vs = {'Y', 'X1', 'X2', 'Zb', 'Zc', 'Za'},
    directed_edges= (('X1', 'Zb'), ('X1', 'Za'), ('X1', 'X2'), ('X1', 'Y'), ('Za', 'X2'), ('Zb', 'Za'), ('Zb', 'Zc'), ('X2', 'Y'), ('Zc', 'Za')) ,
    bidirected_edges = {('Y', 'Zc', 'U3'), ('X1', 'Zb', 'U0'), ('Y', 'Za', 'U2'), ('X1', 'Za', 'U1')}
)
sac = SAC(G_4,{'X1','X2'},{'Y'},{'Za','Zb','Zc'})
sac.construct()
print(sac.Z)
print('SAC criterion :',sac.criteria())

sac.minsac()
print(sac.Z)
print('SAC criterion :',sac.criteria())


[frozenset(), frozenset({'Zc', 'Za', 'Zb'})]
SAC criterion : True
[frozenset(), frozenset({'Za'})]
SAC criterion : True
